**In this notebook we have used the headlines and the main frame of the articles as input and output respectively to train the bert model**

In [ ]:
pip install accelerate -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 333.2/333.2 kB 15.7 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.34.2
    Uninstalling accelerate-0.34.2:
      Successfully uninstalled accelerate-0.34.2


In [ ]:
import pandas as pd
from google.colab import files
upload = files.upload()


Saving Training_Data.csv to Training_Data.csv


In [ ]:
!pip install transformers torch


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from transformers.modeling_outputs import SequenceClassifierOutput
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import numpy as np


In [ ]:
df = pd.read_csv('Training_Data.csv')
df.head()

,title,first_tone_annotation,first_framing_annotation
0,A Set of Borders to Cross; For Children Seekin...,17.35,10.2
1,LAY OFF THE GUEST WORKER WE WANT,17.35,1.2
2,Would ban on renting to illegals make sense here?,19.35,5.2
3,"Immigrants Pull Weight In Economy, Study Finds",17.35,1.2
4,The Citizenship Surge,17.35,2.2


In [ ]:
df = df.dropna(subset=['first_framing_annotation'])

In [ ]:
df = df.dropna(subset=['title'])

In [ ]:
len(df['first_framing_annotation'].unique())

15

In [ ]:
# Shift labels to range from 0 to 14
df['first_framing_annotation'] = df['first_framing_annotation'] - 1


In [ ]:
df['first_framing_annotation'].unique()

array([ 9,  0,  4,  1, 10, 12, 11,  6,  3,  5,  7,  8, 13,  2, 14])

In [ ]:
import pandas as pd
# Convert column values to integers
df['first_framing_annotation'] = df['first_framing_annotation'].astype(int)

headlines = df['title'].tolist()
frames = df['first_framing_annotation'].tolist()


In [ ]:
class NewsDataset(Dataset):
    def __init__(self, headlines, frames, tokenizer, max_length):
        self.headlines = headlines
        self.frames = frames
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.headlines)

    def __getitem__(self, idx):
        text = self.headlines[idx]
        label = self.frames[idx]
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'labels': torch.tensor(label, dtype=torch.long)
        }


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=15)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
class FocalLoss(nn.Module):
    def __init__(self, alpha=None, gamma=2.0):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma

    def forward(self, logits, labels):
        ce_loss = nn.CrossEntropyLoss()(logits, labels)
        pt = torch.exp(-ce_loss)
        focal_loss = (self.alpha * (1 - pt) ** self.gamma * ce_loss).mean()
        return focal_loss


In [ ]:
# Split data
train_texts, val_texts, train_labels, val_labels = train_test_split(headlines, frames, test_size=0.2, random_state=42)

train_dataset = NewsDataset(train_texts, train_labels, tokenizer, max_length=128)
val_dataset = NewsDataset(val_texts, val_labels, tokenizer, max_length=128)

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=4)


In [ ]:
# Set up training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    num_train_epochs=10,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    learning_rate=2e-5,
    weight_decay=0.01,

)

# Custom model with Focal Loss
class CustomBERTModel(nn.Module):
    def __init__(self, model, num_labels, focal_loss):
        super(CustomBERTModel, self).__init__()
        self.model = model
        self.num_labels = num_labels
        self.focal_loss = focal_loss

    def forward(self, input_ids, attention_mask, labels):
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        loss = self.focal_loss(logits, labels)
        return SequenceClassifierOutput(loss=loss, logits=logits)

focal_loss = FocalLoss(alpha=0.25, gamma=2)  # Adjust alpha for class imbalance
custom_model = CustomBERTModel(model, num_labels=15, focal_loss=focal_loss)

trainer = Trainer(
    model=custom_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

# Train model
trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.428500,0.309466
2,0.221100,0.305375
3,0.117500,0.323467
4,0.051400,0.348479
5,0.023500,0.381463
6,0.011400,0.398033
7,0.007200,0.425170
8,0.003900,0.434463
9,0.002500,0.439315
10,0.002500,0.445920


TrainOutput(global_step=9820, training_loss=0.08317298066713173, metrics={'train_runtime': 18686.1284, 'train_samples_per_second': 2.102, 'train_steps_per_second': 0.526, 'total_flos': 0.0, 'train_loss': 0.08317298066713173, 'epoch': 10.0})

In [ ]:
results = trainer.evaluate()
print(results)


{'eval_loss': 0.4459196627140045, 'eval_runtime': 123.0177, 'eval_samples_per_second': 7.983, 'eval_steps_per_second': 2.0, 'epoch': 10.0}


In [ ]:
from sklearn.metrics import accuracy_score
import numpy as np

# Make predictions on the validation dataset
predictions = trainer.predict(val_dataset)
preds = np.argmax(predictions.predictions, axis=1)  # Get the predicted class labels
labels = predictions.label_ids                    # True labels

# Calculate micro accuracy
micro_accuracy = accuracy_score(labels, preds)
print(f"Micro Accuracy: {micro_accuracy:.4f}")


Micro Accuracy: 0.4745
